# Data Wrangling and Parameter Selection


<a id='guide'></a>
# Guide
### 1. Import libraries
### 2. [Create cost of living table](#livingwage)
 - 2a. [Import MIT living wage data by year (html files)](#importmit)
 - 2b. [Clean 2019 table (different format than following years)](#clean2019)
 - 2c. Concatenate 2020 - current year tables and then add 2019 table
 - 2d. [Extract number of adults and children](#extractnumerical)
 - 2e. [Calculate costs by person and category](#personcost)
 - 2f. [Create circumstance cost and disposable unit parameter levels](#parameterlevels)
 - 2g. [Create the cost dataframe](#costdf)
 - 2h. Export cost table file

### 3. Add circumstance columns to fabricated data 
 - 3a. Import fabricated data
 - 3b. [Natural Language Processing of additional circumstances](#nlp)
 - 3c. Export file and manually check for missed information
 
### 4. Parameter Selection
 - 4a. [Set scholarship tiers](#tiers)
 - 4b. [Create long dataframe with all variations](#long)

# 1. Import necessary libraries

In [1]:
# Import the libraries
import requests
import io
import pandas as pd
import numpy as np
import re

_____________________
<a id='livingwage'></a>
# 2. Create cost of living table


<a id='importmit'></a>
## 2a. Import MIT living wage data by year (html files)

In [2]:
# import from GitHub:Floydworks

url_19 = ('https://raw.githubusercontent.com/Floydworks/Scholarship_Allocation_Tool/main/MIT_cost_of_living_by_year/MIT_alameda_2019.html')
download = requests.get(url_19).content
#Read HTML tables into a list of DataFrame objects
df_list19 = pd.read_html(url_19)
req_inc_19 = df_list19[1]
req_inc_19['year'] = '2019'
print(req_inc_19['year'][0])


url_20 = ('https://raw.githubusercontent.com/Floydworks/Scholarship_Allocation_Tool/main/MIT_cost_of_living_by_year/MIT_alameda_2020.html')
download = requests.get(url_20).content
#Read HTML tables into a list of DataFrame objects
df_list20 = pd.read_html(url_20)
req_inc_20 = df_list20[1]
req_inc_20['year'] = '2020'
print(req_inc_20['year'][0])


url_21 = ('https://raw.githubusercontent.com/Floydworks/Scholarship_Allocation_Tool/main/MIT_cost_of_living_by_year/MIT_alameda_2021.html')
download = requests.get(url_21).content
#Read HTML tables into a list of DataFrame objects
df_list21 = pd.read_html(url_21)
req_inc_21 = df_list21[1]
req_inc_21['year'] = '2021'
print(req_inc_21['year'][0])


url_22 = ('https://raw.githubusercontent.com/Floydworks/Scholarship_Allocation_Tool/main/MIT_cost_of_living_by_year/MIT_alameda_2022.html')
download = requests.get(url_22).content
#Read HTML tables into a list of DataFrame objects
df_list22 = pd.read_html(url_22)
req_inc_22 = df_list22[1]
req_inc_22['year'] = '2022'
print(req_inc_22['year'][0])


url_23 = ('https://raw.githubusercontent.com/Floydworks/Scholarship_Allocation_Tool/main/MIT_cost_of_living_by_year/MIT_alameda_2023.html')
download = requests.get(url_23).content
#Read HTML tables into a list of DataFrame objects
df_list23 = pd.read_html(url_23)
req_inc_23 = df_list23[1]
req_inc_23['year'] = '2023'
print(req_inc_23['year'][0])


2019
2020
2021
2022
2023


<a id='clean2019'></a>
## 2b. Clean 2019 table (different format than following years)

In [3]:
#get just the table with 'required annual income before taxes'
full_income_table =  req_inc_19         #  df_list[1]
#transpose table 
table_19 = full_income_table.T
#make first row into column headers
table_19.columns = table_19.iloc[0]
#drop first row of text that is now column headers
table_19 = table_19.iloc[1: , :]
#reset the index, move fam size to column
table_19 = table_19.reset_index()

table_19

Annual Expenses,index,Food,Child Care,Medical,Housing,Transportation,Other,Required annual income after taxes,Annual taxes,Required annual income before taxes
0,1 Adult,"$3,573",$0,"$2,121","$18,480","$4,206","$2,976","$31,356","$4,975","$36,331"
1,1 Adult 1 Child,"$5,267","$8,311","$6,965","$27,948","$7,664","$4,951","$61,105","$10,965","$72,070"
2,1 Adult 2 Children,"$7,929","$13,997","$6,622","$27,948","$9,011","$5,375","$70,882","$13,238","$84,121"
3,1 Adult 3 Children,"$10,517","$19,683","$6,704","$38,628","$10,425","$6,256","$92,214","$17,911","$110,124"
4,2 Adults (1 Working),"$6,551",$0,"$5,271","$22,260","$7,664","$4,951","$46,696","$7,893","$54,589"
5,2 Adults (1 Working) 1 Child,"$8,154",$0,"$6,622","$27,948","$9,011","$5,375","$57,110","$10,060","$67,170"
6,2 Adults (1 Working) 2 Children,"$10,529",$0,"$6,704","$27,948","$10,425","$6,256","$61,863","$11,141","$73,004"
7,2 Adults (1 Working) 3 Children,"$12,820",$0,"$6,423","$38,628","$10,307","$6,121","$74,300","$13,746","$88,046"
8,2 Adults (1 Working Part Time) 1 Child*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$75,760"
9,2 Adults,"$6,551",$0,"$5,271","$22,260","$7,664","$4,951","$46,696","$7,893","$54,589"


In [4]:
print(table_19['index'][1])

#Remove unicode '\xa0' from pandas column if present
table_19['index'] = table_19['index'].str.split().str.join(' ')

print(table_19['index'][1])

1 Adult 1 Child
1 Adult 1 Child


In [5]:
#rename index column to hold count of adults in household
table_19 = table_19.rename(columns={"index": "adults"}, errors="raise")
#make duplicate adults column that will become count for children in household
table_19['children'] = table_19.loc[:, 'adults']
#add year column
table_19['year'] = '2019'
req_inc_19 = table_19[['adults', 'children','Required annual income before taxes', 'year']]
#drop row the has year in it
req_inc_19 = req_inc_19[req_inc_19['adults'].str.contains('year')==False]
#drop row that doesn't match other year formats and is not needed
req_inc_19 = req_inc_19.drop(labels=8, axis=0)


In [6]:
req_inc_19

Annual Expenses,adults,children,Required annual income before taxes,year
0,1 Adult,1 Adult,"$36,331",2019
1,1 Adult 1 Child,1 Adult 1 Child,"$72,070",2019
2,1 Adult 2 Children,1 Adult 2 Children,"$84,121",2019
3,1 Adult 3 Children,1 Adult 3 Children,"$110,124",2019
4,2 Adults (1 Working),2 Adults (1 Working),"$54,589",2019
5,2 Adults (1 Working) 1 Child,2 Adults (1 Working) 1 Child,"$67,170",2019
6,2 Adults (1 Working) 2 Children,2 Adults (1 Working) 2 Children,"$73,004",2019
7,2 Adults (1 Working) 3 Children,2 Adults (1 Working) 3 Children,"$88,046",2019
9,2 Adults,2 Adults,"$54,589",2019
10,2 Adults 1 Child,2 Adults 1 Child,"$77,389",2019


## 2c. Concatenate 2020 - current year tables and then add 2019 table

In [7]:
tables = [req_inc_20, req_inc_21, req_inc_22, req_inc_23] #req_inc_19 removed for different format
tables_df = pd.DataFrame()

for t in tables:
    #get the year
    year = t['year'][0]
    t = t.drop(['year'],axis=1)
    #transpose table 
    df_table = t.T
    #make first row into column headers
    df_table.columns = df_table.iloc[0]
    #drop first row of text that is now column headers
    df_table = df_table.iloc[1: , :]
    df_table['year'] = year
    tables_df = pd.concat([tables_df, df_table])
    #print(df_table)

tables_df.head()

/var/folders/bv/4mctz8d94zx0nlh0hjsm4x8h0000gn/T/ipykernel_41730/1075209967.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  t = t.drop(['year'],axis=1)
/var/folders/bv/4mctz8d94zx0nlh0hjsm4x8h0000gn/T/ipykernel_41730/1075209967.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  t = t.drop(['year'],axis=1)
/var/folders/bv/4mctz8d94zx0nlh0hjsm4x8h0000gn/T/ipykernel_41730/1075209967.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  t = t.drop(['year'],axis=1)
/var/folders/bv/4mctz8d94zx0nlh0hjsm4x8h0000gn/T/ipykernel_41730/1075209967.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  t = t.drop(['year'],axis=1)


('Unnamed: 0_level_0', 'Unnamed: 0_level_1')     Food Child Care Medical  \
1 ADULT             0 Children                 $3,592         $0  $2,211   
                    1 Child                    $5,306     $8,448  $7,364   
                    2 Children                 $7,976    $14,228  $7,076   
                    3 Children                $10,578    $20,007  $7,196   
2 ADULTS(1 WORKING) 0 Children                 $6,586         $0  $5,455   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1')  Housing Transportation   Other  \
1 ADULT             0 Children                $16,908         $4,094  $2,734   
                    1 Child                   $25,512         $7,982  $4,558   
                    2 Children                $25,512        $10,126  $4,732   
                    3 Children                $35,100        $11,032  $5,953   
2 ADULTS(1 WORKING) 0 Children                $20,472         $7,982  $4,558   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') Required annual income after taxes  \
1 ADULT             0 Children                                          $29,540   
                    1 Child                                             $59,170   
                    2 Children                                          $69,650   
                    3 Children                                          $89,865   
2 ADULTS(1 WORKING) 0 Children                                          $45,053   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') Annual taxes  \
1 ADULT             0 Children                     $4,748   
                    1 Child                       $10,856   
                    2 Children                    $13,293   
                    3 Children                    $17,844   
2 ADULTS(1 WORKING) 0 Children                     $7,733   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') Required annual income before taxes  \
1 ADULT             0 Children                                           $34,288   
                    1 Child                                              $70,026   
                    2 Children                                           $82,942   
                    3 Children                                          $107,709   
2 ADULTS(1 WORKING) 0 Children                                           $52,786   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1')  year Civic  
1 ADULT             0 Children                2020   NaN  
                    1 Child                   2020   NaN  
                    2 Children                2020   NaN  
                    3 Children                2020   NaN  
2 ADULTS(1 WORKING) 0 Children                2020   NaN

In [8]:
# Get only the columns needed
df_income_table = pd.DataFrame(tables_df[['Required annual income before taxes', 'year']])
# move index values to columns
df_income_table = df_income_table.reset_index()
df_income_table.columns = ['adults', 'children', 'Required annual income before taxes', 'year']

print(df_income_table.columns)
#df_income_table.head()


Index(['adults', 'children', 'Required annual income before taxes', 'year'], dtype='object')


In [9]:
df_income_table

,adults,children,Required annual income before taxes,year
0,1 ADULT,0 Children,"$34,288",2020
1,1 ADULT,1 Child,"$70,026",2020
2,1 ADULT,2 Children,"$82,942",2020
3,1 ADULT,3 Children,"$107,709",2020
4,2 ADULTS(1 WORKING),0 Children,"$52,786",2020
5,2 ADULTS(1 WORKING),1 Child,"$65,688",2020
6,2 ADULTS(1 WORKING),2 Children,"$71,396",2020
7,2 ADULTS(1 WORKING),3 Children,"$86,442",2020
8,2 ADULTS(BOTH WORKING),0 Children,"$52,786",2020
9,2 ADULTS(BOTH WORKING),1 Child,"$76,105",2020


<a id='extractnumerical'></a>
## 2d. Extract number of adults and children

In [10]:
# define a function to clean up characters, case, and spacing
def text_cleaning(txt):

  # Take care of non-ascii characters
  txt = (txt.encode('ascii','replace')).decode("utf-8")

  # Convert text to Lower case
  txt = txt.lower()

  # Remove multiple spacing
  space = re.sub(r"\s+"," ",txt, flags = re.I)

  # Remove special characters
  clean_text = re.sub(r'[!"#$%&()*+,-./:;?@[\]^_`{|}~]',' ',space)
  clean_text = clean_text.replace(" ", "")
    
  space1 = re.sub(r"\s+"," ",clean_text, flags = re.I) 

  #return print('Output : ', space1)
  return space1

In [11]:
#make empty lists for storing numbers of family members
n_adults = []
n_children = []

# extract number of adults and number of children
for r in df_income_table['adults']:
    text = text_cleaning(str(r))
    #text = re.sub('\s+','',str(r).lower())
    #print(text)
    find_a = text.find("a")
    a = text[find_a - 1]
    #n_adults.append(int(a))
    n_adults.append(a)

for r in df_income_table['children']:
    text = text_cleaning(str(r))
    find_c = text.find("c")
    c = text[find_c - 1]
    #print(a, c)
    #n_children.append(int(c))
    n_children.append(c)

#reset index after concatenation
df_income_table['num_adults'] = n_adults
df_income_table['num_children'] = n_children
df_income_table['req_income_pretax'] = df_income_table['Required annual income before taxes'].astype('str')\
                                       .str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

#get rid of the letters in 2019 in place of 0 children
#define a function that converts letters to '0' in the num_children column
def convert_numb(x, chil):
    if x.isnumeric() != True:
        chil.append('0')
        #print("Not Numeric")
    else:
        chil.append(x)
        #print("numeric")

#get num_children column data in list form
kids = list(df_income_table['num_children'])
#create empty list to store returned values
chld = []
#loop through the list and convert letters to '0'
for k in kids:
    convert_numb(k, chld)

#assign corrected values to the num_children column
df_income_table['num_children'] = chld

#df_income_table.head(60)


In [12]:
df_income_table = df_income_table[['year','num_adults','num_children','req_income_pretax']]
df_income_table.head()

,year,num_adults,num_children,req_income_pretax
0,2020,1,0,34288
1,2020,1,1,70026
2,2020,1,2,82942
3,2020,1,3,107709
4,2020,2,0,52786


<a id='personcosts'></a>
## 2e. Calculate costs by person and category

In [13]:
#get list of years in df
years = list(df_income_table['year'].unique())

#create empty lists for variable storage
yr = []
base = []
add_child = []
add_adult = []
single_cost = []

for y in years:
    #get df for particular year
    df_annual_income_table = df_income_table[df_income_table['year']==y].reset_index()
    
    #pull base income for one adult no kids for that year
    base_annual = df_annual_income_table['req_income_pretax'][0]
    
    # calculate the cost of an additional child with one adult in household
    one_child1_annual = df_annual_income_table['req_income_pretax'][1] - df_annual_income_table['req_income_pretax'][0]
    two_child1_annual = df_annual_income_table['req_income_pretax'][2] - df_annual_income_table['req_income_pretax'][1]
    three_child1_annual = df_annual_income_table['req_income_pretax'][3] - df_annual_income_table['req_income_pretax'][2]
    avg_child1_annual = (one_child1_annual + two_child1_annual + three_child1_annual)/3
    
    # calculate the cost of an additional child with two adults in household
    one_child2_annual = df_annual_income_table['req_income_pretax'][9] - df_annual_income_table['req_income_pretax'][8]
    two_child2_annual = df_annual_income_table['req_income_pretax'][10] - df_annual_income_table['req_income_pretax'][9]
    three_child2_annual = df_annual_income_table['req_income_pretax'][11] - df_annual_income_table['req_income_pretax'][10]
    avg_child2_annual = (one_child2_annual+two_child2_annual+three_child2_annual)/3
    
    # calculate the additional cost per child for a single parent
    single_cost_annual = avg_child1_annual - avg_child2_annual
    
    # calculate cost of additional adult
    add_adult1_annual = df_annual_income_table['req_income_pretax'][9] - df_annual_income_table['req_income_pretax'][1]
    add_adult2_annual = df_annual_income_table['req_income_pretax'][10] - df_annual_income_table['req_income_pretax'][2]
    add_adult3_annual = df_annual_income_table['req_income_pretax'][11] - df_annual_income_table['req_income_pretax'][3]
    avg_adult_annual = (add_adult1_annual+add_adult2_annual+add_adult3_annual)/3
    
    # set values for additional household members
    add_adult_annual = avg_adult_annual
    add_child_annual = avg_child2_annual
    
    #set default value for cost of additional circumstances
    #############circumstance_cost_annual = 5000

    #print some things as a sanity check
    print(y, ': ', base_annual, add_adult_annual.round(2), add_child_annual.round(2),
          single_cost_annual.round(2), len(df_annual_income_table['req_income_pretax']))
    
    #do all the appendings
    yr.append(y)
    base.append(base_annual)
    add_adult.append(add_adult_annual)
    add_child.append(add_child_annual)
    single_cost.append(single_cost_annual)
    ##########circumstance_cost.append(circumstance_cost_annual)  #used list with varying amounts in it by year
    

2020 :  34288 5153.33 19438.0 5035.67 12
2021 :  45520 5407.0 29891.0 8265.67 12
2022 :  50463 7671.33 33949.67 7704.67 12
2023 :  46488 4494.67 35656.0 8753.33 12


<a id='parameterlevels'></a>
## 2f. Create circumstance cost and disposable unit parameter levels

In [14]:
# set the circumstance cost for 2019, 2020, 2021, 2022, 2023
circumstance_cost_dict_low = {'2019':1000, '2020':1000, '2021':2000, '2022':2000, '2023':2000}
circumstance_cost_dict_mid = {'2019':2000, '2020':2000, '2021':3000, '2022':3000, '2023':3000}    

# set value for disposable income unit for 2019, 2020, 2021, 2022, 2023
disposable_unit_dict_low = {'2019':3000, '2020':3000, '2021':5000, '2022':5000, '2023':5000}      
disposable_unit_dict_mid = {'2019':4000, '2020':4000, '2021':6500, '2022':6500, '2023':6500}
disposable_unit_dict_hi = {'2019':5000, '2020':5000, '2021':7000, '2022':7000, '2023':7000}


In [15]:
#create the cost dataframe
cost_df = pd.DataFrame(columns = ['year'])
cost_df['year'] = yr
cost_df['base_income'] = base
cost_df['add_adult'] = add_adult
cost_df['add_child'] = add_child
cost_df['single_cost'] = single_cost
cost_df = cost_df.sort_values(by = 'year').reset_index(drop=True)

#add circumstances and disposable units to cost_df, the cost dataframe
cost_df['circumstance_cost_low'] = cost_df['year'].map(circumstance_cost_dict_low)
cost_df['circumstance_cost_mid'] = cost_df['year'].map(circumstance_cost_dict_mid)

cost_df['disposable_unit_low'] = cost_df['year'].map(disposable_unit_dict_low)
cost_df['disposable_unit_mid'] = cost_df['year'].map(disposable_unit_dict_mid)
cost_df['disposable_unit_hi'] = cost_df['year'].map(disposable_unit_dict_hi)

#print(len(cost_df))
#print(cost_df.columns)
#cost_df.head()

In [16]:
#create long version of cost dataframe, cost_df
#long version of disposable_unit
#cost_df_long2 = pd.melt(cost_df, id_vars=['year', 'base_income', 'add_adult', 'add_child', 'single_cost',
#                                         'circumstance_cost_mid', 'circumstance_cost_low', ], 
#                       value_vars=['disposable_unit_low',
#                                   'disposable_unit_mid', 'disposable_unit_hi'],
#                       var_name=['disposable_level'],
#                       value_name='disposable_value'
#                      )
#print(len(cost_df_long2))
#print(cost_df_long2.columns)
#cost_df_long2.head(5)

#add circumstance levels to long version
#cost_df_long = pd.melt(cost_df_long2, id_vars=['year', 'base_income', 'add_adult', 'add_child', 'single_cost',
#                                               'disposable_level', 'disposable_value'], 
#                       value_vars=['circumstance_cost_mid', 'circumstance_cost_low',],
#                       var_name=['circumstance_level'],
#                       value_name='circumstance_value'
#                      )

#print(len(cost_df_long))
#print(cost_df_long.columns)

#check that long version has correct length and format
#print(len(cost_df))
#print((len(cost_df_long2)), 'value_vars= disposable_unit_low, disposable_unit_mid, disposable_unit_hi')
#print((len(cost_df_long)), 'value_vars= circumstance_cost_mid, circumstance_cost_low')
#cost_df_long.head(26)

In [17]:
cost_df

,year,base_income,add_adult,add_child,single_cost,circumstance_cost_low,circumstance_cost_mid,disposable_unit_low,disposable_unit_mid,disposable_unit_hi
0,2020,34288,5153.333333,19438.000000,5035.666667,1000,2000,3000,4000,5000
1,2021,45520,5407.000000,29891.000000,8265.666667,2000,3000,5000,6500,7000
2,2022,50463,7671.333333,33949.666667,7704.666667,2000,3000,5000,6500,7000
3,2023,46488,4494.666667,35656.000000,8753.333333,2000,3000,5000,6500,7000


## 2h. Export cost table file 

In [18]:
#export file as csv and manually check for accuracy in caluculations
#cost_df.to_csv('/FILE PATH/cost_df.csv')

[Got to Top](#guide)
___________________
<a id='applicantdata'></a>
# 3. Add circumstance columns to fabricated data 

<a id='import'></a>
## 3a. Import fabricated data

In [19]:
#get data from GitHub: Floydworks
url = ('https://raw.githubusercontent.com/Floydworks/Scholarship_Allocation_Tool/main/data_files/fabricated_scholarship_applicant_data.csv')
download = requests.get(url).content
# Read the downloaded content and turn it into a pandas dataframe
df_Fab = pd.read_csv(io.StringIO(download.decode('utf-8')))

df_Fab = df_Fab.rename(columns={"household_income_pre_tax": "household_income", })

df_Fab

,name,applied,household_income,number_adults,number_children,additional_expenses
0,a,yes,0,1,1,NaN
1,b,yes,0,1,1,NaN
2,c,yes,0,3,2,NaN
3,d,yes,0,2,2,NaN
4,e,yes,10000,1,2,NaN
...,...,...,...,...,...,...
98,vvvv,yes,190000,2,4,"education, medical, housing"
99,wwww,yes,200000,2,1,NaN
100,xxxx,yes,200000,3,2,NaN
101,yyyy,yes,220000,2,4,we have two children in college


In [20]:
#export file as csv
#orig_df.to_csv('/FILE PATH/orig_df.csv')

<a id='nlp'></a>
## 3b.  Natural Language Processing of additional circumstances

### Clean text in additional_expenses, remove characters and spaces

In [21]:
# define a function to clean up characters, case, and spacing
def text_cleaning(txt):

  # Take care of non-ascii characters
  txt = (txt.encode('ascii','replace')).decode("utf-8")

  # Convert text to Lower case
  txt = txt.lower()

  # Remove multiple spacing
  space = re.sub(r"\s+"," ",txt, flags = re.I)

  # Remove special characters
  clean_text = re.sub(r'[!"#$%&()*+,-./:;?@[\]^_`{|}~]',' ',space)
  clean_text = clean_text.replace(" ", "")
    
  space1 = re.sub(r"\s+"," ",clean_text, flags = re.I) 

  #return print('Output : ', space1)
  return space1
 

In [22]:
application_data = df_Fab
application_data.head(3)

,name,applied,household_income,number_adults,number_children,additional_expenses
0,a,yes,0,1,1,NaN
1,b,yes,0,1,1,NaN
2,c,yes,0,3,2,NaN


In [23]:
#replace nan values with 'none' string
application_data['additional_expenses'] = application_data['additional_expenses'].replace(np.nan, 'none')


In [24]:
application_data.head(3)

,name,applied,household_income,number_adults,number_children,additional_expenses
0,a,yes,0,1,1,none
1,b,yes,0,1,1,none
2,c,yes,0,3,2,none


### Define searchable phrases for each expense category

In [25]:
#define phrases for additional circumstances in 'Notes' column
divorce = [
#           'recentdivorce',
#           'recentlydivorced'
#           'separated'
          ]
           
medical = ['medical',
           'health',
           'disability',
           'injury',
           'injured',
           'surgery',
           'sick',
           'cancer'
          ]

family = ['elderly',
          'elder',
          'caregiving',
          'caringfor'
         ]
           
education = ['college',
             'school',
             'education',
             'tuition',
             'loan',
             'loans',
             'doctoral',
             'masters',
             'graduatedegree',
             'university'
            ]

employment = ['jobloss',
              'lostjob',
              'lostmyjob',
              'unemploy',
              'unemployment',
              'umemployed',
              'incomeloss',
              'lossofincome',
              'lossinincome',
              'reducedhours',
              'parttime',
              'laidoff',
              'fired',
              'letgo'            
             ]

immigration = ['undocumented',
               'visa',
               'travel',
               'country',
               'immigrant',
               'immigration',
               'livesabroad',
               'overseas',
               'citizen'
              ]

housing = ['foreclosure',
           'therent',
           'ourrent',
           'myrent',
           'rent',
           'highrent',
           'rentisexpensive',
           'expensiverent',
           'moving',
           'moved',
           'repair',
           'bill',
           'bills',
           'housing'
          ]

In [26]:
#initialize various storage arrays
divorce_array =[]
medical_array = []
family_array = []
education_array = []
employment_array = []
immigration_array = []
housing_array = []
single_array = []

df_divorce = []
df_medical = []
df_family = []
df_education = []
df_employment = []
df_immigration = []
df_housing = []
df_single = []

In [27]:
#deep learning
#words that occur in a sequence, find phrases that are similar
#similarity search bert, spacy

### search notes for circumstance phrases as defined above and assign to arrays

In [28]:
application_data.columns

Index(['name', 'applied', 'household_income', 'number_adults',
       'number_children', 'additional_expenses'],
      dtype='object')

In [29]:
application_data

,name,applied,household_income,number_adults,number_children,additional_expenses
0,a,yes,0,1,1,none
1,b,yes,0,1,1,none
2,c,yes,0,3,2,none
3,d,yes,0,2,2,none
4,e,yes,10000,1,2,none
...,...,...,...,...,...,...
98,vvvv,yes,190000,2,4,"education, medical, housing"
99,wwww,yes,200000,2,1,none
100,xxxx,yes,200000,3,2,none
101,yyyy,yes,220000,2,4,we have two children in college


In [30]:
#single parent
for i in application_data['number_adults']:
    if i == 1:
        single_array.append(1)
    else:
        single_array.append(0)
    df_single = single_array
        
#divorce
for i in application_data['additional_expenses']:
    text1 = text_cleaning(str(i))
    text2 = text1.replace(' ', '')
    for t in divorce:
            if t in text2:
                #print('divorce added')
                divorce_array.append(1)
            else:
                divorce_array.append(0)
    df_divorce.append(sum(divorce_array))
    divorce_array = []

    
#employment
for i in application_data['additional_expenses']:
    text1 = text_cleaning(str(i))
    text2 = text1.replace(' ', '')
    for t in employment:
            if t in text2:
                #print('employment added')
                employment_array.append(1)
            else:
                employment_array.append(0)
    df_employment.append(sum(employment_array))
    employment_array = []    

    
#medical
for i in application_data['additional_expenses']:
    text1 = text_cleaning(str(i))
    text2 = text1.replace(' ', '')
    for t in medical:
            if t in text2:
                #print('medical added')
                medical_array.append(1)
            else:
                medical_array.append(0)
    df_medical.append(sum(medical_array))
    medical_array = []  

#family
for i in application_data['additional_expenses']:
    text1 = text_cleaning(str(i))
    text2 = text1.replace(' ', '')
    for t in family:
            if t in text2:
               #print('family added')
               family_array.append(1)
            else:
                family_array.append(0)
    df_family.append(sum(family_array))
    family_array = []  

    
#education
for i in application_data['additional_expenses']:    
    text1 = text_cleaning(str(i))
    text2 = text1.replace(' ', '')
    for t in education:
            if t in text2:
                #print('education added')
                education_array.append(1)
            else:
                education_array.append(0)
    df_education.append(sum(education_array))
    education_array = []      

    
#immigration
for i in application_data['additional_expenses']:
    text1 = text_cleaning(str(i))
    text2 = text1.replace(' ', '')
    for t in immigration:
            if t in text2:
                #print('immigration added')
                immigration_array.append(1)
            else:
                immigration_array.append(0)
    df_immigration.append(sum(immigration_array))
    immigration_array = []  

    
#housing
for i in application_data['additional_expenses']:
    text1 = text_cleaning(str(i))
    text2 = text1.replace(' ', '')
    for t in housing:
            if t in text2:
                #print('housing added')
                housing_array.append(1)
            else:
                housing_array.append(0)
    df_housing.append(sum(housing_array))
    housing_array = []  
    

In [31]:
application_data.head()

,name,applied,household_income,number_adults,number_children,additional_expenses
0,a,yes,0,1,1,none
1,b,yes,0,1,1,none
2,c,yes,0,3,2,none
3,d,yes,0,2,2,none
4,e,yes,10000,1,2,none


### create table of circumstances for each household

In [32]:
#initialize empty dataframe for results
circumstances = pd.DataFrame()
#create new feature columns
circumstances['employment'] = df_employment
circumstances['divorce'] = df_divorce
circumstances['medical'] = df_medical
circumstances['education'] = df_education
circumstances['immigration'] = df_immigration
circumstances['housing'] = df_housing

#store column names
cols = circumstances.columns
#if same circumstance is mentioned twice or more, change values above 0 to 1
circumstances = pd.DataFrame(np.where(circumstances > 0, 1, 0))
#reassign column names
circumstances.columns = cols
#create column summing the number of addtional circumstances
circumstances['total_circumstances'] = circumstances.sum(axis=1)

circumstances['single'] = df_single # added value is per child
circumstances['family'] = df_family # flagged for checking

#circumstances

In [33]:
#concat original data and new features dataframes
df_Fab_train = pd.concat([application_data, circumstances], axis = 1)
#df_Fab_train.head(2)

In [34]:
df_Fab_train.columns

Index(['name', 'applied', 'household_income', 'number_adults',
       'number_children', 'additional_expenses', 'employment', 'divorce',
       'medical', 'education', 'immigration', 'housing', 'total_circumstances',
       'single', 'family'],
      dtype='object')

## 3c. Export file and manually check for missed information

In [35]:
#export file as csv and manually check for accuracy in caluculations

#df_Fab_train.to_csv('/LOCAL FILE PATH/df.csv')


In [36]:
df_Fab_train

,name,applied,household_income,number_adults,number_children,additional_expenses,employment,divorce,medical,education,immigration,housing,total_circumstances,single,family
0,a,yes,0,1,1,none,0,0,0,0,0,0,0,1,0
1,b,yes,0,1,1,none,0,0,0,0,0,0,0,1,0
2,c,yes,0,3,2,none,0,0,0,0,0,0,0,0,0
3,d,yes,0,2,2,none,0,0,0,0,0,0,0,0,0
4,e,yes,10000,1,2,none,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,vvvv,yes,190000,2,4,"education, medical, housing",0,0,1,1,0,1,3,0,0
99,wwww,yes,200000,2,1,none,0,0,0,0,0,0,0,0,0
100,xxxx,yes,200000,3,2,none,0,0,0,0,0,0,0,0,0
101,yyyy,yes,220000,2,4,we have two children in college,0,0,0,1,0,0,1,0,0


In [37]:
df_Fab_train.columns

Index(['name', 'applied', 'household_income', 'number_adults',
       'number_children', 'additional_expenses', 'employment', 'divorce',
       'medical', 'education', 'immigration', 'housing', 'total_circumstances',
       'single', 'family'],
      dtype='object')

In [38]:
df_Fab_train['year'] = 2022
df_Fab_train = df_Fab_train[['name', 'applied', 'year', 'household_income', 'number_adults',
       'number_children', 'additional_expenses', 'employment', 'divorce',
       'medical', 'education', 'immigration', 'housing', 'total_circumstances',
       'single', 'family']]
df_Fab_train.head(3)

,name,applied,year,household_income,number_adults,number_children,additional_expenses,employment,divorce,medical,education,immigration,housing,total_circumstances,single,family
0,a,yes,2022,0,1,1,none,0,0,0,0,0,0,0,1,0
1,b,yes,2022,0,1,1,none,0,0,0,0,0,0,0,1,0
2,c,yes,2022,0,3,2,none,0,0,0,0,0,0,0,0,0


In [39]:
#export file as csv and manually check for accuracy in caluculations
#df_Fab_train.to_csv('/FILE PATH/fabricated_circumstances.csv')


# 4. Parameter Selection

<a id='tiers'></a>
## 4a. set scholarship tiers

In [40]:
#define scholarship levels
tier1 = 90   # <--- Adjustable parameter: upper scholarship tier, most need
tier2 = 80   # <--- Adjustable parameter
tier3 = 70   # <--- Adjustable parameter
tier4 = 50   # <--- Adjustable parameter
tier5 = 0      


In [41]:
cost_df.head()

,year,base_income,add_adult,add_child,single_cost,circumstance_cost_low,circumstance_cost_mid,disposable_unit_low,disposable_unit_mid,disposable_unit_hi
0,2020,34288,5153.333333,19438.000000,5035.666667,1000,2000,3000,4000,5000
1,2021,45520,5407.000000,29891.000000,8265.666667,2000,3000,5000,6500,7000
2,2022,50463,7671.333333,33949.666667,7704.666667,2000,3000,5000,6500,7000
3,2023,46488,4494.666667,35656.000000,8753.333333,2000,3000,5000,6500,7000


In [42]:
cost_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   4 non-null      object 
 1   base_income            4 non-null      int64  
 2   add_adult              4 non-null      float64
 3   add_child              4 non-null      float64
 4   single_cost            4 non-null      float64
 5   circumstance_cost_low  4 non-null      int64  
 6   circumstance_cost_mid  4 non-null      int64  
 7   disposable_unit_low    4 non-null      int64  
 8   disposable_unit_mid    4 non-null      int64  
 9   disposable_unit_hi     4 non-null      int64  
dtypes: float64(3), int64(6), object(1)
memory usage: 448.0+ bytes


In [43]:
df_Fab_train.head()

,name,applied,year,household_income,number_adults,number_children,additional_expenses,employment,divorce,medical,education,immigration,housing,total_circumstances,single,family
0,a,yes,2022,0,1,1,none,0,0,0,0,0,0,0,1,0
1,b,yes,2022,0,1,1,none,0,0,0,0,0,0,0,1,0
2,c,yes,2022,0,3,2,none,0,0,0,0,0,0,0,0,0
3,d,yes,2022,0,2,2,none,0,0,0,0,0,0,0,0,0
4,e,yes,2022,10000,1,2,none,0,0,0,0,0,0,0,1,0


<a id='long'></a>
## 4b. Create long dataframe with all variations

<br>**base_income** = basic needs income for one adult with no child in Alameda County
<br>**add_adult** = cost per each additional adult
<br>**add_child** = cost per each additional child
<br>**single_parent_cost** = additional cost per child for single parent, derived from MIT values for required income adult with a child - required income adult with no child
<br>**circumstance_cost** = SET PARAMETER; cost of each additional circumstance (medical, tuition, etc.)
<br>**extra_cost** = number of circumstances mutiplied by the set annual circumstance cost. Extra cost 
<br>**family_base_income** = base income needed for total number adults/children; base+add_child+add_adult 
<br>**family_need_income** = family_base_income+extra_cost; base need plus cost of additional circumstances
<br>**family_disposable_unit** = SET PARAMETER; choosen unit/bucket of disposable unit per family member, used as disposable unit multiplier for scholarship tier.
<br>**disposable_income** = disposable income above basic needs, when a family has more than 10k in disposable income per adult AND more than 20k in dosposable income per child, the scholarship award falls to zero.

In [44]:
# make additional df_fab dataframe for manipulation
df_fab = df_Fab_train
df_fab['year']=2022

In [45]:
#create empty dataframe for appending
df_fab_awards = pd.DataFrame()

#get list of years to iterate over
years = list(df_fab['year'].unique())

#initiate combo assignment
combo = 0

for y in years:
    #get applicant df for particular year
    df_annual_past_apps = df_fab[df_fab['year']==y].reset_index(drop=True)
    
    #cost dataframe
    df_cost_annual = cost_df[cost_df['year']==str(y)].reset_index(drop=True)
    base = (df_cost_annual.iloc[0][1])
    add_adult = (df_cost_annual.iloc[0][2])
    add_child = (df_cost_annual.iloc[0][3])
    single_cost = (df_cost_annual.iloc[0][4])
    
    # single parent additional expense, whole = full cost per child, half = whole cost first child + half cost each additional child
    single_cost_half = ((single_cost + (df_annual_past_apps['number_children']-1)*(single_cost/2))*df_annual_past_apps['single'])
    single_cost_whole = df_annual_past_apps['number_children']*single_cost*df_annual_past_apps['single']
    #single_parent_cost_vector = [single_cost_half, single_cost_whole]
    single_parent_cost_dict = {'half':single_cost_half, 'whole':single_cost_whole}
    
    #get circumstance_cost by level and create vector/list
    circumstance_cost_low = (df_cost_annual.iloc[0][6])
    circumstance_cost_mid = (df_cost_annual.iloc[0][5])
    #circumstance_vector = [ circumstance_cost_low,  circumstance_cost_mid]
    circumstance_dict = {'low':circumstance_cost_low,  'mid':circumstance_cost_mid}
    
    #get disposable_unit  costs by level and create vector/list
    disposable_unit_low = (df_cost_annual.iloc[0][7])
    disposable_unit_mid = (df_cost_annual.iloc[0][8])
    disposable_unit_hi = (df_cost_annual.iloc[0][9])
    #disposable_unit_vector = [disposable_unit_low, disposable_unit_mid, disposable_unit_hi]
    disposable_unit_dict = {'low':disposable_unit_low, 'mid':disposable_unit_mid, 'hi':disposable_unit_hi}
    
    #reported income (this should include approx income received from other adults not in household)
    income = df_annual_past_apps['household_income']
    
    #estimate income needed for additional adults and children
    family_base_income = base + ((df_annual_past_apps['number_adults']-1)*add_adult) + ((df_annual_past_apps['number_children'])*add_child)
    df_annual_past_apps['family_base_income'] = round(family_base_income, 2)
    
    # loop through ciscumstance_cost levels
    for kc, vc in circumstance_dict.items():
        df_annual_past_apps['circumstance_cost_text'] = kc
        df_annual_past_apps['circumstance_cost'] = vc
        
        # additional loop for single_cost, half or whole
        for ks,vs in single_parent_cost_dict.items():
            df_annual_past_apps['single_cost_text'] = ks
            df_annual_past_apps['single_cost'] = vs
            
            #get sum on additional circumstances like college, medical, moving, loss etc. and multiple by circumstance cost, add any single parent need
            extra_cost = (df_annual_past_apps['total_circumstances']*vc) + vs
            df_annual_past_apps['extra_cost'] = round(extra_cost, 2)
    
            #estimate income needed with extra circumstances costs
            family_need_income = family_base_income + extra_cost
            df_annual_past_apps['family_need_income'] = round(family_need_income, 2)
    
            #disposable income column
            #df_annual_past_apps['ex_inc'] = income - threshold_income
            df_annual_past_apps['disposable_income'] = round(income - family_need_income, 2)
            
            #loop through disposable unit levels
            for kd, vd in disposable_unit_dict.items():
                df_annual_past_apps['disposable_unit_text'] = kd
                df_annual_past_apps['disposable_unit'] = vd
                #create additional support column: disposable income units, adults = 0.5, children = 1
                df_annual_past_apps['family_disposable_unit'] = (df_annual_past_apps['number_adults']*(vd/2)) + (df_annual_past_apps['number_children']*vd)
    
                #initialize and empty list for storing results
                eligibility = []

                #determine eleigibility based on criteria settings
                for e,s in zip(df_annual_past_apps['disposable_income'], df_annual_past_apps['family_disposable_unit']):
    
                    if e < 0:                             #disposable income is below 0: "tier 1" = 90%
                        eligibility.append(tier1)             
                    elif (e > 0 and (e - (s * 1)) < 0):   #disposable income is greater than zero, but less than set value: tier 2 = 80%
                        eligibility.append(tier2)             
                    elif (e > 0 and (e - (s * 1.5)) < 0): #disposable income is greater than the set value, but less than 1.5 times: tier 3 = 70%
                        eligibility.append(tier3)             
                    elif (e > 0 and (e - (s * 2)) < 0):   #disposable income is less than twice the set value: tier 4 = 50%
                        eligibility.append(tier4)             
                    else:                                 #disposable income is twice the set value or more: tier 5 = 0%
                        eligibility.append(tier5)

                #assign the results array as a new column, 'percent_eligible'
                df_annual_past_apps['percent_eligible'] = eligibility
                
                combo = combo+1
                #assign the results array as a new column, 'percent_eligible'
                df_annual_past_apps['combination'] = str(combo)

                ##concatenate 2019
                df_fab_awards = pd.concat([df_annual_past_apps, df_fab_awards], axis = 0)

#print(df_annual_past_apps.columns)
#df_annual_past_apps.head(3)

In [46]:
df_fab_awards_long = df_fab_awards.reset_index(drop=True)
df_fab_awards_long.head(50)

,name,applied,year,household_income,number_adults,number_children,additional_expenses,employment,divorce,medical,...,single_cost_text,single_cost,extra_cost,family_need_income,disposable_income,disposable_unit_text,disposable_unit,family_disposable_unit,percent_eligible,combination
0,a,yes,2022,0,1,1,none,0,0,0,...,whole,7704.666667,7704.67,92117.33,-92117.33,hi,7000,10500.0,90,12
1,b,yes,2022,0,1,1,none,0,0,0,...,whole,7704.666667,7704.67,92117.33,-92117.33,hi,7000,10500.0,90,12
2,c,yes,2022,0,3,2,none,0,0,0,...,whole,0.000000,0.00,133705.00,-133705.00,hi,7000,24500.0,90,12
3,d,yes,2022,0,2,2,none,0,0,0,...,whole,0.000000,0.00,126033.67,-126033.67,hi,7000,21000.0,90,12
4,e,yes,2022,10000,1,2,none,0,0,0,...,whole,15409.333333,15409.33,133771.67,-123771.67,hi,7000,17500.0,90,12
5,f,yes,2022,15500,1,1,I have an injury that is keeping me from work,0,0,1,...,whole,7704.666667,9704.67,94117.33,-78617.33,hi,7000,10500.0,90,12
6,g,yes,2022,26800,1,1,our housing costs have gone up substantially,0,0,0,...,whole,7704.666667,9704.67,94117.33,-67317.33,hi,7000,10500.0,90,12
7,h,yes,2022,27000,12,3,none,0,0,0,...,whole,0.000000,0.00,236696.67,-209696.67,hi,7000,63000.0,90,12
8,i,yes,2022,30000,2,2,none,0,0,0,...,whole,0.000000,0.00,126033.67,-96033.67,hi,7000,21000.0,90,12
9,j,yes,2022,30000,3,1,none,0,0,0,...,whole,0.000000,0.00,99755.33,-69755.33,hi,7000,17500.0,90,12


## **Selected algorithm parameters:**
<br>Single cost = half (whole for first child, half for every additional child)
<br>Circumstance cost = low
<br>Disposable unit = high